In [19]:
import pandas as pd
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

print("Cuda available", torch.cuda.is_available())  # Should return True if CUDA is available
print("Cuda version", torch.version.cuda)         # Should print the CUDA version PyTorch was built with
print("Device count", torch.cuda.device_count())

Cuda available True
Cuda version 12.1
Device count 1


In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    raise EnvironmentError("CUDA is not available. Check your installation and try again.")

In [ ]:
df1 = pd.read_csv('training_data/QnA_intent_processed_amazon_reviews_us_Beauty.tsv', sep= '\t')
df2 = pd.read_csv('training_data/QnA_intent_processed_amazon_reviews_us_Health.tsv', sep= '\t')


print(df1.shape, df2.shape)

df = pd.concat([df1,df2], ignore_index= True, axis=0)

X = df[['review_id','QnA']]
y = df.verified_purchase
X_train, X_test, _, _ = train_test_split(X, y, random_state= 56, test_size= 0.4) 

X_train.to_csv('training_data/train.tsv', sep= '\t', index= False)
X_test.to_csv('training_data/test.tsv', sep= '\t', index= False)

In [4]:
X_train = pd.read_csv('training_data/train.tsv', sep= '\t')
X_train.shape

(1078930, 2)

In [5]:
class DataFrameDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        qa_pair = str(self.dataframe.iloc[idx]['QnA'])
        encoding = self.tokenizer.encode_plus(qa_pair, return_tensors='pt', max_length=self.max_length, truncation=True, padding='max_length')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        return {'input_ids': input_ids, 'attention_mask': attention_mask}

In [12]:
training_args = TrainingArguments(
    output_dir='./gpt2-finetuned',
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    logging_dir= './logs',
    save_steps=10_000,
    save_total_limit= 2,
    learning_rate= 0.01,
    logging_steps= 1000
)

In [7]:
model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
max_length = 128

dataset = DataFrameDataset(X_train, tokenizer, max_length)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [ ]:
# chunks = np.array_split(X_train, 100)

for idx, chunk in tqdm(enumerate(chunks_2)):
    # if idx == 0:
    #     path = model_name
    # else:
    path = './gpt2-finetuned'
    dataset = DataFrameDataset(chunk, tokenizer, max_length)
    model = GPT2LMHeadModel.from_pretrained(path).to(device)
    model.resize_token_embeddings(len(tokenizer))

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    trainer.train()

    trainer.save_model('./gpt2-finetuned')

0it [00:00, ?it/s]

Step,Training Loss
1000,4.564400
2000,4.549700
3000,4.434600
4000,4.221300
5000,4.171900


1it [27:00, 1620.45s/it]

Step,Training Loss
1000,4.583100
2000,4.594300
3000,4.412400
4000,4.253000
5000,4.192100


2it [1:31:29, 2943.24s/it]

...

Step,Training Loss
1000,4.145600
2000,4.110100
3000,3.953700
4000,3.757300
5000,3.688500


50it [38:26:32, 2867.82s/it]

Step,Training Loss
1000,4.184100
2000,4.108900
3000,3.958600
4000,3.724900
5000,3.667800


51it [38:58:27, 2581.96s/it]

Step,Training Loss
1000,4.127700
2000,4.050900


In [23]:
chunks_2 = chunks[46:]

107893